# Week 4 - MinHash

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

Working with sets, minhash, and comparing sequences.

</div>

## Setup

Make sure you have `python >= 3.10`. Check using `python --version` or `python3 --version`.

**Week 4 packages**

This week we will use:
- biopython
- ipykernel
- seaborn


**Week 4 data**

This week we will be comparing entire bacterial genomes for similarity using MinHash. <br>
The 4 genomes we will use are in the `./data` folder within `week4/`.


## Working with Sets

In Python sets are an unordered collection of unique elements. Sets are similar to lists and tuples, but unlike lists and tuples, sets cannot contain duplicate values. 

We can use sets for tasks that involve handling unique items, such as removing duplicates from a list or testing membership in a collection.

In [ ]:
# Create sets in Python
empty_set = set()
my_set = {3, 4, 5, 12, 123}
my_set

In [ ]:
# Convert a list to a set
set([4, 3, 2, 1, 4, 4])

In [ ]:
# Convert a set to a list, order not guaranteed
list(my_set)

Adding and Removing Elements:

You can add elements to a set using the add() method and remove elements using the remove() or discard() methods.

Note: discard() does not raise an error if if the element is not found

In [ ]:
my_set.add(6)
my_set.remove(3)
print(my_set)

In [ ]:
# Membership Testing: You can test if an element is in a set using the in keyword.
if 6 in my_set:
    print("6 is in the set")

In [ ]:
# You can get the number of elements in a set using the len() function.
len(my_set)

In [ ]:
# You can get the minimum element in a set using the min() function.
min(my_set)

### Set Operations:

Sets support various operations such as union, intersection, difference, and symmetric difference.

Union (|): Combines two sets and returns a new set with all unique elements.

In [ ]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}
union_set = set1 | set2
print(union_set)

For python <= 3.9, you will have to use the 'union()' function, rather than the '|' operator.
 
You can use .union(), .intersection(), .difference(), and .symmetric_difference() instead of '|', '&', '-' and '^'. 

In [ ]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}
union_set = set1.union(set2)
print(union_set)

Intersection (&): Returns a new set containing elements that are in both sets.

In [ ]:
intersection_set = set1 & set2
print(intersection_set)

Difference (-): Returns a new set with elements from the first set that are not in the second set.

Also called the "exclusion"

In [ ]:
# In set1 and not in set2
difference_set = set1 - set2
print(difference_set)

Symmetric Difference (^): Returns a new set with elements that are in either of the sets but not in both.\
(Similar to exclusive or, XOR)

In [ ]:
# Unique to either set, not shared
symmetric_difference_set = set1 ^ set2
print(symmetric_difference_set)

## Exercise 1: True Jaccard Similarity Coefficient (Jaccard Index)

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Calculate jaccard for kmer sets. The Jaccard similarity is the `intersection / union`.
    
- Input: Two kmer sets
- Output: Jaccard dist of the sets
    
</div>

In [ ]:
def jaccard(a: set, b: set) -> float:
    # YOUR CODE HERE
    raise NotImplementedError

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"></div>

In [ ]:
# Test your Jaccard function
set1 = set([1, 2, 3, 4, 5])
set2 = set([4, 5, 6, 7, 8])
print(jaccard(set1, set2))    # should equal 0.25

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Write a function that returns a set of all unique kmers in a sequence. 
    
- Input: 
    - A DNA string or Seq object
    - kmer len `k`
- Extract kmers from the input seq
- Output: Return set of all kmers
    
</div>

In [ ]:
def extract_kmers(seq: str, k: int) -> set:
    """
    Extracts all kmers & returns as set.
    Hint: To add new_item to my_set: my_set.add(new_item)
    """
    # YOUR CODE HERE 
    raise NotImplementedError

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"></div>

In [ ]:
# Test your function
seq = 'AGTACGGT'
# should equal {'TACG', 'GTAC', 'CGGT', 'ACGG', 'AGTA'}, order doesn't matter
print(extract_kmers(seq, 4))

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
    
<b>Challange:</b> Combine the above functions to return jaccard index for two sequences and a kmer size k
    
</div>


In [ ]:
def true_jaccard_similarity(seqA: str, seqB: str, k: int) -> float:
    # YOUR CODE HERE 
    raise NotImplementedError

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"></div>

In [ ]:
# identical
seqA = 'AGTACGGTCAGTAGTCGC'
seqB = 'AGTACGGTCAGTAGTCGC'
j = true_jaccard_similarity(seqA, seqB, 4)
print(f'identical sequences true jaccard: {j:.2f}')   # should equal 1.0

# single mismatch 
seqA = 'AGTACGGTAAGTAGTCGC'
seqB = 'AGTACGGTTAGTAGTCGC'
j = true_jaccard_similarity(seqA, seqB, 4)
print(f'single mismatch true jaccard: {j:.2f}')       # should equal 0.59

# single indel
seqA = 'AGTACGGTAAAGTAGTCGC'
seqB = 'AGTACGGTAAGTAGTCGC'
j = true_jaccard_similarity(seqA, seqB, 4) 
print(f'single indel jaccard: {j:.2f}')               # should equal 0.81


## Exercise 2: MinHash sketches

In exercise 1 we compared kmer similarity between two sequences by calculating a ***true*** jaccard index.  <br>
All kmers from each sequences were used. This can be time consuming if we are comparing lots of sequences.

To reduce runtime, we can create a 'MinHash sketch', which acts as a kmer 'fingerprint' of a sequence. <br>
By comparing these sketches (which are tiny in comparison), rather than the full kmer sets, we can ***estimate*** the jaccard index in a fraction of the time.

**Note:** When comparing only two sequences, MinHash will not reduce runtime.


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Write a function that returns a Minhash Sketch of a DNA sequence. The sketch should be a set containing the bottom `s` hash values of kmers extracted from the input sequence`. Use python default hash() function.

- Input: 
    - A DNA string
    - kmer len `k`
    - the number of min. hash values `s` to store in the sketch 
- Output: Return a set of the smallest `s` hash values

Process:
- Calculate hash values using hash()
- Pick the smallest `s` values and return as a set

Hint:
- You can sort a list using sorted() function.
    
</div>

In [ ]:
def minhash_sketch(seq: str, k: int, s: int) -> set:
    """
    Calulate minhash sketch from DNA sequence.
    """
    # YOUR CODE HERE 
    raise NotImplementedError

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"></div>

In [ ]:
# First 100 bases of our first SeqRecord
seq = 'AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC'
minhash_sketch(seq, k = 5, s = 10)

## Exercise 3: MinHash Jaccard Similarity

Genomes that share kmers also share kmer hash values, and therefore may have similar MinHash sketches. <br>
We can rapidly ***estimate*** the similarity of multiple sequences by calculating the Jaccard similarity using MinHash sketches as input.

For demonstration purpose, we'll be comparing only two sequences. In reality, we should use MinHash only when comparing multiple sequences.

Remember these similarity are estimates only, as we've taken a kmer sample from both sequences.

Using our minhash_sketch() function from earlier, let's test it on two identical sequences.

In [ ]:
# identical sequences: prove that minhash works & jaccard would be 1
seqA = 'AGTACGGTAGATGCGTTGTGCATGACTGATGCTAGAGTCTGCTACGTAGCGACAGCTTGCAGTCATGC'
seqB = 'AGTACGGTAGATGCGTTGTGCATGACTGATGCTAGAGTCTGCTACGTAGCGACAGCTTGCAGTCATGC'

sketchA = minhash_sketch(seqA, 4, 8)
print(f'sketchA: {sketchA}')

sketchB = minhash_sketch(seqB, 4, 8)
print(f'sketchB: {sketchB}')

In [ ]:
# We can check these are the same using an assert statement
assert sketchA == sketchB

Now lets make the sequences slightly different

In [ ]:
# Different sequences: show that most min 8 hash values match
seqA = 'AGTACGGTACATCCGTTGGGC'
seqB = 'AGTACGGTACATGCGTTGC'

sketchA = minhash_sketch(seqA, 4, 8)
sketchB = minhash_sketch(seqB, 4, 8)

if sketchA != sketchB:
    print('sketches not same!')
    
print(f'sketchA: {sketchA}')
print(f'sketchB: {sketchB}')

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<b>Challange:</b> Calculate the Jaccard similarity between two Minhash sketches (Minhash similarity)
    
- Input: 
    - Two sets DNA seqs
    - kmer len k
    - Number of minimizers m
- Calculate minhash sketch for each input seq
- Return: Jaccard similarity between the two minhash sketches
</div>

In [ ]:
def minhash_jaccard_similarity(seqA: str, seqB: str, k: int, s: int) -> float:
    # YOUR CODE HERE 
    raise NotImplementedError

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"></div>

In [ ]:
# identical sequences 
print('IDENTICAL')
seqA = 'AGTACGGTACATGCGTTGC'
seqB = 'AGTACGGTACATGCGTTGC'
jt = true_jaccard_similarity(seqA, seqB, k=4)
jm = minhash_jaccard_similarity(seqA, seqB, k=4, s=10)
print(f'True Jaccard for identical sequences: {jt:.2f}')       # should equal 1.00
print(f'MinHash Jaccard for identical sequences: {jm:.2f}')    # should approximate 1.00
print()

# different sequences: 
# - might get different values for minhash jaccard based on hash seed
print('DIFFERENT')
seqA = 'AGTACGGTAGATGCGTTGTGCATGACTGATGCTAGAGTCTGCTACGTAGCGACAGCTTGCAGTCATGC'
seqB = 'AGTACGGTACATGCGTTGTGCACGACTGATGCTAGAGTCTGCTACGTAGCGACAGCTTGCAAGTCATGC'
jt = true_jaccard_similarity(seqA, seqB, k=4)
jm = minhash_jaccard_similarity(seqA, seqB, k=4, s=10)
print(f'True Jaccard for different sequences: {jt:.2f}')      # should equal 0.77
print(f'MinHash Jaccard for different sequences: {jm:.2f}')   # should approximate 0.77

## Exercise 4: Minimizer Jaccard Similarity

MinHash is useful when comparing lots of sequences, but it doesn't improve the speed or memory cost for two long sequences. For a long sequence, extract all the kmers can be time and memory consuming.

**Minimizer:** Select the `minimum kmer` from a `window` according to `a deterministic order`. Windows should overlap by k-1 to extract kmers at boundaries.

In this exercise, we will use lexicographical order.
**Note:** lexicographical order might be prone to GC bias. Thus, use a MinHash order can be a better choice.

Load bacterial genomes from file using BioPython.

In [ ]:
# load bacterial genomes
import gzip
from Bio import SeqIO

genome_filepaths = [
    'data/NC_000913.fasta.gz',
    'data/NC_002695.fasta.gz',
    'data/NC_003197.fasta.gz',
    'data/NC_021870.fasta.gz',
]
genomes = []
for filepath in genome_filepaths:
    with gzip.open(filepath, "rt") as fp:
        seq = next(SeqIO.parse(fp, "fasta"))
    genomes.append(seq)

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Write a function that returns a Minimizer Sketch of a DNA sequence. The sketch should be a set containing the lexicographically minimum kmer in each window of the input sequence.

- Input: 
    - A DNA string
    - kmer len `k`
    - The number of minimizer kmers `s` to store in the sketch
- Output: Return a set of minimizer kmers with size `s`

Process:
- Calculate the stride size according to sequence length and sketch size `s`
- Extract the kmers in each window, and pick the lexicographically minimum one
- Add the picked kmers to the set, and return it

Hint:
- Ensure windows are overlapped by k-1
- Assume sequence length $>>$ `k` $*$ `s`
- Use `extract_kmers(seq, k)` to extract a set of kmers
    
</div>

In [ ]:
def minimizer_sketch(seq: str, k: int, s: int) -> set:
    """
    Calulate minimizer sketch from DNA sequence.
    """
    seq_len = len(seq)
    stride = seq_len // s
    # Ensure windows overlap by k - 1
    window_size = stride + k - 1
    minimizer_set = set()

    # YOUR CODE HERE 
    raise NotImplementedError

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;"></div>

In [ ]:
seqA = genomes[0].seq[:1000000]
minimizer_sketch(seqA, k=10, s=20)
seB = genomes[1].seq[:1000000]
minimizer_sketch(seB, k=10, s=20)

You may notice that the above cell runs quicker than the true Jaccard similarity cell.

Now it's time to calculate the estimated Jaccard similarity using minimizers.

In [ ]:
def minimizer_jaccard_similarity(seqA: str, seqB: str, k: int, s: int) -> float:

    sketchA = minimizer_sketch(seqA, k, s)
    sketchB = minimizer_sketch(seqB, k, s)
    j = jaccard(set(sketchA), set(sketchB))
    return j

In [ ]:
# identical sequences 
print('IDENTICAL')
seqA = genomes[0].seq[:1000000]
seqB = seqA
jt = true_jaccard_similarity(seqA, seqB, k=37)
jm = minimizer_jaccard_similarity(seqA, seqB, k=37, s=3000)
print(f'True Jaccard for identical sequences: {jt:.2f}')      # should equal 1.00
print(f'Minimizer Jaccard for identical sequences: {jm:.2f}')    # should equal 1.00, as deterministic
print()

# different sequences: 
print('DIFFERENT')
seqA = genomes[0].seq[:1000000]
seqB = genomes[1].seq[:1000000]
jt = true_jaccard_similarity(seqA, seqB, k=37)
jm = minimizer_jaccard_similarity(seqA, seqB, k=37, s=5000)
print(f'True Jaccard for different sequences: {jt:.2f}')
print(f'Minimizer Jaccard for different sequences: {jm:.2f}')

## Extension: Speed and accuracy comparison

Here we demonstrate the speed improvement and accuracy tradeoff when using MinHash and Minimizer. 

Calculate ***true*** Jaccard similarity (using all kmers). 
Only using the first 1Mb of a given genome. 

In [ ]:
seqA = genomes[0].seq[:1000000]
seqB = genomes[1].seq[:1000000]
print('true jaccard index: ', true_jaccard_similarity(seqA, seqB, k=37))

The cell above should take a few seconds - half a minute to run. 

This is pretty slow if we want to do many comparisons. <br>
Let's instead extract MinHash sketches, then compare sketches instead. 

Extract sketches (a few seconds - half a minute to run)

In [ ]:
sketch_A = minhash_sketch(genomes[0].seq[:1000000], 37, 1000)
sketch_B = minhash_sketch(genomes[1].seq[:1000000], 37, 1000)

After sketches have been created, comparisons will now take negligible time. <br>
Below we will calculate the jaccard index 100 times to illustrate. It should still run almost instantly. 

Note that the ***estimated*** jaccard index is very similar to the true jaccard index. 

In [ ]:
for i in range(100):
    jaccard(set(sketch_A), set(sketch_B))

print('MinHash estimated jaccard index: ', jaccard(set(sketch_A), set(sketch_B)))

From this example we see that the estimated jaccard index is close to the true value. <br> 
MinHash can provide an efficiency boost while maintaining decent accuracy. <br>
MinHash (and other fingerprinting approaches) can be applied when doing one-to-many or many-to-many comparisons. 

Minimizer really shines when the sequence is long enough to overcome the randomness. We compare the two whole genome, and analyze the runtime differences. Following code might take several minutes to run depending on the hardware.

In [ ]:
kmer_setA = extract_kmers(genomes[0].seq, k=37)
kmer_setB = extract_kmers(genomes[1].seq, k=37)
print('true jaccard index: ', jaccard(kmer_setA, kmer_setB))

Sketching the two genomes should take less than half the time.

In [ ]:
sketchA = minimizer_sketch(genomes[0].seq, 37, 8000)
sketchB = minimizer_sketch(genomes[1].seq, 37, 8000)

Comparing the sketches also takes negligible time.

In [ ]:
print('Minimizer estimated jaccard index: ', jaccard(sketchA, sketchB))

The estimation is not as good as MinHash. However, the memory consumption really makes a difference.

In [ ]:
import sys
true_mem = sys.getsizeof(kmer_setA) + sys.getsizeof(kmer_setB)
minimizer_mem = sys.getsizeof(sketchA) + sys.getsizeof(sketchA)
print("True Jaccard consumes: ", true_mem, "memory")
print("Minimizer consumes", true_mem, "memory")
print("Difference:", true_mem / minimizer_mem, "times")